In [ ]:
import numpy as np
import pandas as pd
import phonenumbers
import re
from datetime import datetime


EXTRACTION

In [188]:
customer_raw=pd.read_csv('customer_raw.csv')
product_raw=pd.read_csv('products_raw.csv')
sales_raw=pd.read_csv('sales_raw.csv')


In [189]:
customer_raw.head()

,customer_id,first_name,last_name,email,phone,city,registration_date
0,C001,Rahul,Sharma,rahul.sharma@gmail.com,9876543210,Bangalore,2023-01-15
1,C002,Priya,Patel,priya.patel@yahoo.com,+91-9988776655,Mumbai,2023-02-20
2,C003,Amit,Kumar,NaN,9765432109,Delhi,2023-03-10
3,C004,Sneha,Reddy,sneha.reddy@gmail.com,9123456789,Hyderabad,15/04/2023
4,C005,Vikram,Singh,vikram.singh@outlook.com,09988112233,Chennai,2023-05-22


In [179]:
product_raw.head()

,product_id,product_name,category,price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,fashion,3499.0,80.0
2,P003,Apple MacBook Pro,ELECTRONICS,NaN,45.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,electronics,1999.0,200.0


In [180]:
sales_raw.head()

,transaction_id,customer_id,product_id,quantity,unit_price,transaction_date,status
0,T001,C001,P001,1,45999.0,2024-01-15,Completed
1,T002,C002,P004,2,2999.0,2024-01-16,Completed
2,T003,C003,P007,1,52999.0,15/01/2024,Completed
3,T004,NaN,P002,1,3499.0,2024-01-18,Pending
4,T005,C005,P009,3,650.0,2024-01-20,Completed


TRANSFORMATION

In [190]:
# def find_treat_missing_values(df, subset_cols=None):
#     df = df.copy()
#     missing_col = []

#     for col, cnt in df.isna().sum().items():
#         if cnt > 0:
#             missing_col.append(f'{col} has {cnt} missing records')

#     df.dropna(subset=subset_cols, inplace=True)

#     return df, missing_col



def find_treat_missing_values(df): 
    missing_col=[] 
    numerical_col=df.select_dtypes(include=['int64','float64']).columns.tolist() 
    categorical_cols=df.select_dtypes(include=['object','category','bool']).columns.tolist() 
    for col,cnt in df.isna().sum().items(): 
        if cnt>0: 
            missing_col.append(f'{col} has {cnt} missing records') 
        else: 
            df.dropna(subset=[col],inplace=True) 
    return df, missing_col

In [ ]:
customer,missing_col= find_treat_missing_values(customer_raw)

In [192]:
missing_col

['email has 5 missing records']

In [255]:
customer.dropna(subset=['email'], inplace=True)

In [256]:
customer

,customer_id,first_name,last_name,email,city,registration_date,std_phone,customer_sk
0,C025,Sanjay,Agarwal,sanjay.agarwal@gmail.com,Lucknow,2024-03-28,+91-9876543276,1
2,C022,Priyanka,Jain,priyanka.jain@yahoo.com,Indore,2024-03-15,+91-9876543287,3
3,C021,Nikhil,Bose,nikhil.bose@gmail.com,Kolkata,2024-03-10,+91-9988776600,4
4,C020,Swati,Desai,swati.desai@gmail.com,Pune,2024-03-01,+91-9123456712,5
6,C017,Rajesh,Kumar,rajesh.kumar@gmail.com,Delhi,2024-02-12,+91-9876123450,7
7,C016,Divya,Menon,divya.menon@gmail.com,Bangalore,2024-02-05,+91-9123456701,8
8,C014,Neha,Shah,neha.shah@gmail.com,Ahmedabad,2024-01-15,+91-9876543221,9
9,C013,Suresh,Patel,suresh.patel@outlook.com,Mumbai,2024-01-08,+91-9123409876,10
11,C011,Arjun,Rao,arjun.rao@gmail.com,Hyderabad,2023-11-05,+91-9876509876,12
12,C010,Deepa,Gupta,deepa.gupta@gmail.com,Delhi,2023-10-12,+91-9871234567,13


In [194]:
def standardize_phone_numbers(df, phone_col='phone', new_col='std_phone'):
    df = df.copy()

    def extract_last_10(phone):
        if pd.isna(phone):
            return None
        digits = re.sub(r'\D', '', str(phone))
        return digits[-10:] if len(digits) >= 10 else None

    df[new_col] = df[phone_col].apply(extract_last_10)
    return df

In [ ]:


customer = (
    standardize_phone_numbers(customer, phone_col='phone', new_col='std_phone')
    .sort_values(by='registration_date', ascending=False)  # latest first
    .drop_duplicates(subset=['std_phone'], keep='first')   # keep latest
    .reset_index(drop=True)
)

In [187]:
customer

,customer_id,first_name,last_name,email,phone,city,registration_date,std_phone
0,C025,Sanjay,Agarwal,sanjay.agarwal@gmail.com,09876543276,Lucknow,2024-03-28,9876543276
1,C023,Rohit,Kapoor,NaN,9988112211,Chandigarh,2024-03-20,9988112211
2,C022,Priyanka,Jain,priyanka.jain@yahoo.com,9876543287,Indore,2024-03-15,9876543287
3,C021,Nikhil,Bose,nikhil.bose@gmail.com,+919988776600,Kolkata,2024-03-10,9988776600
4,C020,Swati,Desai,swati.desai@gmail.com,9123456712,Pune,2024-03-01,9123456712
5,C018,Kavya,Reddy,NaN,9988112200,Hyderabad,2024-02-18,9988112200
6,C017,Rajesh,Kumar,rajesh.kumar@gmail.com,09876123450,Delhi,2024-02-12,9876123450
7,C016,Divya,Menon,divya.menon@gmail.com,9123456701,Bangalore,2024-02-05,9123456701
8,C014,Neha,Shah,neha.shah@gmail.com,+91-9876543221,Ahmedabad,2024-01-15,9876543221
9,C013,Suresh,Patel,suresh.patel@outlook.com,9123409876,Mumbai,2024-01-08,9123409876


In [196]:
customer.drop(columns=['phone'], inplace=True)

In [197]:
customer

,customer_id,first_name,last_name,email,city,registration_date,std_phone
0,C025,Sanjay,Agarwal,sanjay.agarwal@gmail.com,Lucknow,2024-03-28,9876543276
1,C023,Rohit,Kapoor,NaN,Chandigarh,2024-03-20,9988112211
2,C022,Priyanka,Jain,priyanka.jain@yahoo.com,Indore,2024-03-15,9876543287
3,C021,Nikhil,Bose,nikhil.bose@gmail.com,Kolkata,2024-03-10,9988776600
4,C020,Swati,Desai,swati.desai@gmail.com,Pune,2024-03-01,9123456712
5,C018,Kavya,Reddy,NaN,Hyderabad,2024-02-18,9988112200
6,C017,Rajesh,Kumar,rajesh.kumar@gmail.com,Delhi,2024-02-12,9876123450
7,C016,Divya,Menon,divya.menon@gmail.com,Bangalore,2024-02-05,9123456701
8,C014,Neha,Shah,neha.shah@gmail.com,Ahmedabad,2024-01-15,9876543221
9,C013,Suresh,Patel,suresh.patel@outlook.com,Mumbai,2024-01-08,9123409876


In [198]:
# Add country code prefix
customer['std_phone'] = '+91-' + customer['std_phone']

In [199]:

product_raw.rename(columns={'price': 'unit_price'}, inplace=True)

In [200]:
product_raw

,product_id,product_name,category,unit_price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,fashion,3499.0,80.0
2,P003,Apple MacBook Pro,ELECTRONICS,NaN,45.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,electronics,1999.0,200.0
5,P006,Organic Almonds,Groceries,899.0,NaN
6,P007,HP Laptop,Electronics,52999.0,60.0
7,P008,Adidas T-Shirt,FASHION,1299.0,150.0
8,P009,Basmati Rice 5kg,groceries,650.0,300.0
9,P010,OnePlus Nord,Electronics,NaN,95.0


In [211]:
sales=sales_raw.merge(product_raw, how='left', on='unit_price')

In [212]:
sales

,transaction_id,customer_id,product_id_x,quantity,unit_price,transaction_date,status,product_id_y,product_name,category,stock_quantity
0,T001,C001,P001,1,45999.0,2024-01-15,Completed,P001,Samsung Galaxy S21,Electronics,150.0
1,T002,C002,P004,2,2999.0,2024-01-16,Completed,P004,Levi's Jeans,Fashion,120.0
2,T003,C003,P007,1,52999.0,15/01/2024,Completed,P007,HP Laptop,Electronics,60.0
3,T004,NaN,P002,1,3499.0,2024-01-18,Pending,P002,Nike Running Shoes,fashion,80.0
4,T005,C005,P009,3,650.0,2024-01-20,Completed,P009,Basmati Rice 5kg,groceries,300.0
5,T006,C006,P012,1,12999.0,01-22-2024,Completed,P012,Dell Monitor 24inch,Electronics,40.0
6,T007,C007,P005,2,1999.0,2024-01-23,Completed,P005,Sony Headphones,electronics,200.0
7,T008,C008,NaN,1,1299.0,2024-01-25,Completed,P008,Adidas T-Shirt,FASHION,150.0
8,T009,C009,P011,1,4599.0,2024-01-28,Cancelled,P011,Puma Sneakers,Fashion,70.0
9,T010,C010,P006,5,899.0,2024-02-01,Completed,P006,Organic Almonds,Groceries,NaN


In [213]:
sales.drop(columns=['product_id_x','product_name','category','stock_quantity'], inplace=True)

In [214]:
sales

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y
0,T001,C001,1,45999.0,2024-01-15,Completed,P001
1,T002,C002,2,2999.0,2024-01-16,Completed,P004
2,T003,C003,1,52999.0,15/01/2024,Completed,P007
3,T004,NaN,1,3499.0,2024-01-18,Pending,P002
4,T005,C005,3,650.0,2024-01-20,Completed,P009
5,T006,C006,1,12999.0,01-22-2024,Completed,P012
6,T007,C007,2,1999.0,2024-01-23,Completed,P005
7,T008,C008,1,1299.0,2024-01-25,Completed,P008
8,T009,C009,1,4599.0,2024-01-28,Cancelled,P011
9,T010,C010,5,899.0,2024-02-01,Completed,P006


In [215]:
# standardize category names

product_raw['category'] = product_raw['category'].str.title()


In [216]:
product_raw

,product_id,product_name,category,unit_price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,Fashion,3499.0,80.0
2,P003,Apple MacBook Pro,Electronics,NaN,45.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,Electronics,1999.0,200.0
5,P006,Organic Almonds,Groceries,899.0,NaN
6,P007,HP Laptop,Electronics,52999.0,60.0
7,P008,Adidas T-Shirt,Fashion,1299.0,150.0
8,P009,Basmati Rice 5kg,Groceries,650.0,300.0
9,P010,OnePlus Nord,Electronics,NaN,95.0


In [217]:
product_raw.dropna(subset=['unit_price'], inplace=True)

In [278]:
product.dropna(subset=['stock_quantity'], inplace=True)

In [218]:
product_raw

,product_id,product_name,category,unit_price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,Fashion,3499.0,80.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,Electronics,1999.0,200.0
5,P006,Organic Almonds,Groceries,899.0,NaN
6,P007,HP Laptop,Electronics,52999.0,60.0
7,P008,Adidas T-Shirt,Fashion,1299.0,150.0
8,P009,Basmati Rice 5kg,Groceries,650.0,300.0
10,P011,Puma Sneakers,Fashion,4599.0,70.0
11,P012,Dell Monitor 24inch,Electronics,12999.0,40.0


In [201]:

def convert_to_yyyy_mm_dd(date_value):
    if pd.isna(date_value):
        return None

    # If already a datetime object
    if isinstance(date_value, (datetime, pd.Timestamp)):
        return date_value.strftime('%Y-%m-%d')

    # Try parsing common date formats
    date_formats = [
        '%d-%m-%Y',
        '%m-%d-%Y',
        '%Y-%m-%d',
        '%d/%m/%Y',
        '%m/%d/%Y',
        '%Y/%m/%d'
    ]

    for fmt in date_formats:
        try:
            return datetime.strptime(str(date_value), fmt).strftime('%Y-%m-%d')
        except ValueError:
            continue

    # If no format matches
    return None

In [202]:
customer['registration_date'] = customer['registration_date'].apply(convert_to_yyyy_mm_dd)

In [203]:
customer

,customer_id,first_name,last_name,email,city,registration_date,std_phone
0,C025,Sanjay,Agarwal,sanjay.agarwal@gmail.com,Lucknow,2024-03-28,+91-9876543276
1,C023,Rohit,Kapoor,NaN,Chandigarh,2024-03-20,+91-9988112211
2,C022,Priyanka,Jain,priyanka.jain@yahoo.com,Indore,2024-03-15,+91-9876543287
3,C021,Nikhil,Bose,nikhil.bose@gmail.com,Kolkata,2024-03-10,+91-9988776600
4,C020,Swati,Desai,swati.desai@gmail.com,Pune,2024-03-01,+91-9123456712
5,C018,Kavya,Reddy,NaN,Hyderabad,2024-02-18,+91-9988112200
6,C017,Rajesh,Kumar,rajesh.kumar@gmail.com,Delhi,2024-02-12,+91-9876123450
7,C016,Divya,Menon,divya.menon@gmail.com,Bangalore,2024-02-05,+91-9123456701
8,C014,Neha,Shah,neha.shah@gmail.com,Ahmedabad,2024-01-15,+91-9876543221
9,C013,Suresh,Patel,suresh.patel@outlook.com,Mumbai,2024-01-08,+91-9123409876


In [219]:
product= product_raw[product_raw['unit_price'].notna()]

In [220]:
product

,product_id,product_name,category,unit_price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,Fashion,3499.0,80.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,Electronics,1999.0,200.0
5,P006,Organic Almonds,Groceries,899.0,NaN
6,P007,HP Laptop,Electronics,52999.0,60.0
7,P008,Adidas T-Shirt,Fashion,1299.0,150.0
8,P009,Basmati Rice 5kg,Groceries,650.0,300.0
10,P011,Puma Sneakers,Fashion,4599.0,70.0
11,P012,Dell Monitor 24inch,Electronics,12999.0,40.0


In [221]:
sales['transaction_date'] = sales['transaction_date'].apply(convert_to_yyyy_mm_dd)

In [222]:
sales

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y
0,T001,C001,1,45999.0,2024-01-15,Completed,P001
1,T002,C002,2,2999.0,2024-01-16,Completed,P004
2,T003,C003,1,52999.0,2024-01-15,Completed,P007
3,T004,NaN,1,3499.0,2024-01-18,Pending,P002
4,T005,C005,3,650.0,2024-01-20,Completed,P009
5,T006,C006,1,12999.0,2024-01-22,Completed,P012
6,T007,C007,2,1999.0,2024-01-23,Completed,P005
7,T008,C008,1,1299.0,2024-01-25,Completed,P008
8,T009,C009,1,4599.0,2024-01-28,Cancelled,P011
9,T010,C010,5,899.0,2024-02-01,Completed,P006


In [204]:
def clean_extra_space(df, col='product_name'):
    df = df.copy()
    df[col] = (
        df[col]
        .astype(str)
        .str.replace(r'\s+', ' ', regex=True)
        .str.strip()
    )
    return df

In [205]:
product = clean_extra_space(product, col='product_name')
customer = clean_extra_space(customer, col='first_name')
customer = clean_extra_space(customer, col='last_name')

In [208]:
def add_surrogate_key(df, key_col='surrogate_id', start=1):
    df = df.copy().reset_index(drop=True)
    df[key_col] = range(start, start + len(df))
    return df

In [ ]:

customer = add_surrogate_key(customer, key_col='customer_sk')



In [277]:
product = add_surrogate_key(product, key_col='product_sk')

In [284]:
sales


,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y,sales_sk
0,T001,C001,1,45999.0,2024-01-15,Completed,P001,1
1,T002,C002,2,2999.0,2024-01-16,Completed,P004,2
2,T003,C003,1,52999.0,2024-01-15,Completed,P007,3
3,T004,NaN,1,3499.0,2024-01-18,Pending,P002,4
4,T005,C005,3,650.0,2024-01-20,Completed,P009,5
5,T006,C006,1,12999.0,2024-01-22,Completed,P012,6
6,T007,C007,2,1999.0,2024-01-23,Completed,P005,7
7,T008,C008,1,1299.0,2024-01-25,Completed,P008,8
8,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,9
9,T010,C010,5,899.0,2024-02-01,Completed,P006,10


In [259]:
customer

,customer_id,first_name,last_name,email,city,registration_date,std_phone,customer_sk
0,C025,Sanjay,Agarwal,sanjay.agarwal@gmail.com,Lucknow,2024-03-28,+91-9876543276,1
1,C022,Priyanka,Jain,priyanka.jain@yahoo.com,Indore,2024-03-15,+91-9876543287,2
2,C021,Nikhil,Bose,nikhil.bose@gmail.com,Kolkata,2024-03-10,+91-9988776600,3
3,C020,Swati,Desai,swati.desai@gmail.com,Pune,2024-03-01,+91-9123456712,4
4,C017,Rajesh,Kumar,rajesh.kumar@gmail.com,Delhi,2024-02-12,+91-9876123450,5
5,C016,Divya,Menon,divya.menon@gmail.com,Bangalore,2024-02-05,+91-9123456701,6
6,C014,Neha,Shah,neha.shah@gmail.com,Ahmedabad,2024-01-15,+91-9876543221,7
7,C013,Suresh,Patel,suresh.patel@outlook.com,Mumbai,2024-01-08,+91-9123409876,8
8,C011,Arjun,Rao,arjun.rao@gmail.com,Hyderabad,2023-11-05,+91-9876509876,9
9,C010,Deepa,Gupta,deepa.gupta@gmail.com,Delhi,2023-10-12,+91-9871234567,10


In [224]:
product

,product_id,product_name,category,unit_price,stock_quantity
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0
1,P002,Nike Running Shoes,Fashion,3499.0,80.0
3,P004,Levi's Jeans,Fashion,2999.0,120.0
4,P005,Sony Headphones,Electronics,1999.0,200.0
5,P006,Organic Almonds,Groceries,899.0,NaN
6,P007,HP Laptop,Electronics,52999.0,60.0
7,P008,Adidas T-Shirt,Fashion,1299.0,150.0
8,P009,Basmati Rice 5kg,Groceries,650.0,300.0
10,P011,Puma Sneakers,Fashion,4599.0,70.0
11,P012,Dell Monitor 24inch,Electronics,12999.0,40.0


In [ ]:
product,missing_col_prod= find_treat_missing_values(product)

In [ ]:
missing_col_prod

['unit_price has 3 missing records', 'stock_quantity has 1 missing records']

In [321]:
import mysql.connector

# Connect to MySQL
conn = mysql.connector.connect(
    host="localhost",
    user="root",
    password="password",
    database="fleximart"
)

cursor = conn.cursor()


In [273]:
sql = """
        INSERT INTO customers (first_name, last_name, email, phone, city, registration_date)
        VALUES (%s, %s, %s, %s, %s, %s)
    """
values = []
for cust in customer.to_dict(orient='records'):
    values.append((cust["first_name"], cust["last_name"], cust["email"], cust["std_phone"], cust["city"], cust["registration_date"]))

if values:
    cursor.executemany(sql, values)
    conn.commit()
    print(cursor.rowcount, "rows inserted.")
else:
    print("No records to insert.")


19 rows inserted.


In [280]:
product

,product_id,product_name,category,unit_price,stock_quantity,product_sk
0,P001,Samsung Galaxy S21,Electronics,45999.0,150.0,1
1,P002,Nike Running Shoes,Fashion,3499.0,80.0,2
2,P004,Levi's Jeans,Fashion,2999.0,120.0,3
3,P005,Sony Headphones,Electronics,1999.0,200.0,4
4,P007,HP Laptop,Electronics,52999.0,60.0,5
5,P008,Adidas T-Shirt,Fashion,1299.0,150.0,6
6,P009,Basmati Rice 5kg,Groceries,650.0,300.0,7
7,P011,Puma Sneakers,Fashion,4599.0,70.0,8
8,P012,Dell Monitor 24inch,Electronics,12999.0,40.0,9
9,P013,Woodland Shoes,Fashion,5499.0,55.0,10


In [281]:

for prod in product.to_dict(orient='records'):
    sql = """
        INSERT INTO products (product_name, category, price, stock_quantity)
        VALUES (%s,%s, %s, %s)
    """
    value = (prod['product_name'], prod['category'], prod['unit_price'], prod['stock_quantity'])
    cursor.execute(sql, value)
    conn.commit()
    print(cursor.rowcount, prod['product_name'], "inserted.")

1 Samsung Galaxy S21 inserted.
1 Nike Running Shoes inserted.
1 Levi's Jeans inserted.
1 Sony Headphones inserted.
1 HP Laptop inserted.
1 Adidas T-Shirt inserted.
1 Basmati Rice 5kg inserted.
1 Puma Sneakers inserted.
1 Dell Monitor 24inch inserted.
1 Woodland Shoes inserted.
1 iPhone 13 inserted.
1 Organic Honey 500g inserted.
1 Samsung TV 43inch inserted.
1 Masoor Dal 1kg inserted.
1 Boat Earbuds inserted.
1 Reebok Trackpants inserted.


In [285]:
sales.dropna(subset=['customer_id'], inplace=True)


In [295]:
sales = add_surrogate_key(sales, key_col='sales_sk')

In [289]:
sales.drop_duplicates(subset=['transaction_id'], keep='last', inplace=True)

In [290]:
sales

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y,sales_sk
1,T002,C002,2,2999.0,2024-01-16,Completed,P004,2
2,T003,C003,1,52999.0,2024-01-15,Completed,P007,3
3,T005,C005,3,650.0,2024-01-20,Completed,P009,4
4,T006,C006,1,12999.0,2024-01-22,Completed,P012,5
5,T007,C007,2,1999.0,2024-01-23,Completed,P005,6
6,T008,C008,1,1299.0,2024-01-25,Completed,P008,7
7,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,8
8,T010,C010,5,899.0,2024-02-01,Completed,P006,9
9,T001,C001,1,45999.0,2024-01-15,Completed,P001,10
10,T011,C011,1,69999.0,2024-02-02,Completed,P014,11


In [292]:
sales.sort_values(by='transaction_id', ascending=True)

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y,sales_sk
9,T001,C001,1,45999.0,2024-01-15,Completed,P001,10
1,T002,C002,2,2999.0,2024-01-16,Completed,P004,2
2,T003,C003,1,52999.0,2024-01-15,Completed,P007,3
3,T005,C005,3,650.0,2024-01-20,Completed,P009,4
4,T006,C006,1,12999.0,2024-01-22,Completed,P012,5
5,T007,C007,2,1999.0,2024-01-23,Completed,P005,6
6,T008,C008,1,1299.0,2024-01-25,Completed,P008,7
7,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,8
8,T010,C010,5,899.0,2024-02-01,Completed,P006,9
10,T011,C011,1,69999.0,2024-02-02,Completed,P014,11


In [293]:
sales.drop(columns=['sales_sk'], inplace=True)

In [296]:
sales

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y,sales_sk
0,T002,C002,2,2999.0,2024-01-16,Completed,P004,1
1,T003,C003,1,52999.0,2024-01-15,Completed,P007,2
2,T005,C005,3,650.0,2024-01-20,Completed,P009,3
3,T006,C006,1,12999.0,2024-01-22,Completed,P012,4
4,T007,C007,2,1999.0,2024-01-23,Completed,P005,5
5,T008,C008,1,1299.0,2024-01-25,Completed,P008,6
6,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,7
7,T010,C010,5,899.0,2024-02-01,Completed,P006,8
8,T001,C001,1,45999.0,2024-01-15,Completed,P001,9
9,T011,C011,1,69999.0,2024-02-02,Completed,P014,10


In [337]:
order=sales.merge(
        customer,
        how='left',
        on='customer_id'
    )

In [338]:
order

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,product_id_y,sales_sk,first_name,last_name,email,city,registration_date,std_phone,customer_sk
0,T002,C002,2,2999.0,2024-01-16,Completed,P004,1,Priya,Patel,priya.patel@yahoo.com,Mumbai,2023-02-20,+91-9988776655,14.0
1,T003,C003,1,52999.0,2024-01-15,Completed,P007,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,T005,C005,3,650.0,2024-01-20,Completed,P009,3,Vikram,Singh,vikram.singh@outlook.com,Chennai,2023-05-22,+91-9988112233,13.0
3,T006,C006,1,12999.0,2024-01-22,Completed,P012,4,Anjali,Mehta,anjali.mehta@gmail.com,Bangalore,2023-06-18,+91-9876543210,12.0
4,T007,C007,2,1999.0,2024-01-23,Completed,P005,5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,T008,C008,1,1299.0,2024-01-25,Completed,P008,6,Pooja,Iyer,pooja.iyer@gmail.com,Bangalore,2023-08-15,+91-9123456780,17.0
6,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,7,Karthik,Nair,karthik.nair@yahoo.com,Kochi,2023-09-30,+91-9988776644,11.0
7,T010,C010,5,899.0,2024-02-01,Completed,P006,8,Deepa,Gupta,deepa.gupta@gmail.com,Delhi,2023-10-12,+91-9871234567,10.0
8,T001,C001,1,45999.0,2024-01-15,Completed,P001,9,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,T011,C011,1,69999.0,2024-02-02,Completed,P014,10,Arjun,Rao,arjun.rao@gmail.com,Hyderabad,2023-11-05,+91-9876509876,9.0


In [339]:
order.drop(columns=['product_id_y', 'sales_sk', 'first_name','last_name','email','city','std_phone'], inplace=True)

In [340]:
order

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,registration_date,customer_sk
0,T002,C002,2,2999.0,2024-01-16,Completed,2023-02-20,14.0
1,T003,C003,1,52999.0,2024-01-15,Completed,NaN,NaN
2,T005,C005,3,650.0,2024-01-20,Completed,2023-05-22,13.0
3,T006,C006,1,12999.0,2024-01-22,Completed,2023-06-18,12.0
4,T007,C007,2,1999.0,2024-01-23,Completed,NaN,NaN
5,T008,C008,1,1299.0,2024-01-25,Completed,2023-08-15,17.0
6,T009,C009,1,4599.0,2024-01-28,Cancelled,2023-09-30,11.0
7,T010,C010,5,899.0,2024-02-01,Completed,2023-10-12,10.0
8,T001,C001,1,45999.0,2024-01-15,Completed,NaN,NaN
9,T011,C011,1,69999.0,2024-02-02,Completed,2023-11-05,9.0


In [341]:
type(order['customer_sk'])

pandas.core.series.Series

In [344]:
order['customer_sk'] = order['customer_sk'].astype(int)

In [343]:
order.dropna(subset=['customer_sk'],inplace=True)

In [345]:
order

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,registration_date,customer_sk
0,T002,C002,2,2999.0,2024-01-16,Completed,2023-02-20,14
2,T005,C005,3,650.0,2024-01-20,Completed,2023-05-22,13
3,T006,C006,1,12999.0,2024-01-22,Completed,2023-06-18,12
5,T008,C008,1,1299.0,2024-01-25,Completed,2023-08-15,17
6,T009,C009,1,4599.0,2024-01-28,Cancelled,2023-09-30,11
7,T010,C010,5,899.0,2024-02-01,Completed,2023-10-12,10
9,T011,C011,1,69999.0,2024-02-02,Completed,2023-11-05,9
11,T013,C013,3,450.0,2024-02-08,Completed,2024-01-08,8
12,T014,C014,2,1499.0,2024-10-02,Completed,2024-01-15,7
13,T015,C015,3,1299.0,2024-02-12,Completed,2024-01-20,15


In [346]:
order['total_amount']=order['quantity']* order['unit_price']

In [347]:
order

,transaction_id,customer_id,quantity,unit_price,transaction_date,status,registration_date,customer_sk,total_amount
0,T002,C002,2,2999.0,2024-01-16,Completed,2023-02-20,14,5998.0
2,T005,C005,3,650.0,2024-01-20,Completed,2023-05-22,13,1950.0
3,T006,C006,1,12999.0,2024-01-22,Completed,2023-06-18,12,12999.0
5,T008,C008,1,1299.0,2024-01-25,Completed,2023-08-15,17,1299.0
6,T009,C009,1,4599.0,2024-01-28,Cancelled,2023-09-30,11,4599.0
7,T010,C010,5,899.0,2024-02-01,Completed,2023-10-12,10,4495.0
9,T011,C011,1,69999.0,2024-02-02,Completed,2023-11-05,9,69999.0
11,T013,C013,3,450.0,2024-02-08,Completed,2024-01-08,8,1350.0
12,T014,C014,2,1499.0,2024-10-02,Completed,2024-01-15,7,2998.0
13,T015,C015,3,1299.0,2024-02-12,Completed,2024-01-20,15,3897.0


In [348]:
order.drop(columns=['unit_price','quantity'], inplace=True)

In [349]:
order

,transaction_id,customer_id,transaction_date,status,registration_date,customer_sk,total_amount
0,T002,C002,2024-01-16,Completed,2023-02-20,14,5998.0
2,T005,C005,2024-01-20,Completed,2023-05-22,13,1950.0
3,T006,C006,2024-01-22,Completed,2023-06-18,12,12999.0
5,T008,C008,2024-01-25,Completed,2023-08-15,17,1299.0
6,T009,C009,2024-01-28,Cancelled,2023-09-30,11,4599.0
7,T010,C010,2024-02-01,Completed,2023-10-12,10,4495.0
9,T011,C011,2024-02-02,Completed,2023-11-05,9,69999.0
11,T013,C013,2024-02-08,Completed,2024-01-08,8,1350.0
12,T014,C014,2024-10-02,Completed,2024-01-15,7,2998.0
13,T015,C015,2024-02-12,Completed,2024-01-20,15,3897.0


In [350]:
order = add_surrogate_key(order, key_col='order_sk')

In [351]:
order

,transaction_id,customer_id,transaction_date,status,registration_date,customer_sk,total_amount,order_sk
0,T002,C002,2024-01-16,Completed,2023-02-20,14,5998.0,1
1,T005,C005,2024-01-20,Completed,2023-05-22,13,1950.0,2
2,T006,C006,2024-01-22,Completed,2023-06-18,12,12999.0,3
3,T008,C008,2024-01-25,Completed,2023-08-15,17,1299.0,4
4,T009,C009,2024-01-28,Cancelled,2023-09-30,11,4599.0,5
5,T010,C010,2024-02-01,Completed,2023-10-12,10,4495.0,6
6,T011,C011,2024-02-02,Completed,2023-11-05,9,69999.0,7
7,T013,C013,2024-02-08,Completed,2024-01-08,8,1350.0,8
8,T014,C014,2024-10-02,Completed,2024-01-15,7,2998.0,9
9,T015,C015,2024-02-12,Completed,2024-01-20,15,3897.0,10


In [352]:
order.isna().sum()

transaction_id       0
customer_id          0
transaction_date     0
status               0
registration_date    0
customer_sk          0
total_amount         0
order_sk             0
dtype: int64

In [358]:
for ord in order.to_dict(orient='records'):
    sql = """
        INSERT INTO orders (customer_id, order_date, total_amount, status)
        VALUES (%s, %s, %s, %s)
    """
    value = (ord['customer_sk'], ord['registration_date'], ord['total_amount'], ord['status'])
    cursor.execute(sql, value)
    conn.commit()
    print(cursor.rowcount, ord['customer_sk'], "inserted.")

1 14 inserted.
1 13 inserted.
1 12 inserted.
1 17 inserted.
1 11 inserted.
1 10 inserted.
1 9 inserted.
1 8 inserted.
1 7 inserted.
1 15 inserted.
1 5 inserted.
1 19 inserted.
1 4 inserted.
1 3 inserted.
1 14 inserted.
1 16 inserted.
1 13 inserted.
1 12 inserted.
1 17 inserted.
1 11 inserted.
1 9 inserted.
1 8 inserted.
1 7 inserted.
1 15 inserted.
1 6 inserted.
1 5 inserted.
1 19 inserted.
1 4 inserted.


In [353]:
type(order)

pandas.core.frame.DataFrame

In [357]:
order

,transaction_id,customer_id,transaction_date,status,registration_date,customer_sk,total_amount,order_sk
0,T002,C002,2024-01-16,Completed,2023-02-20,14,5998.0,1
1,T005,C005,2024-01-20,Completed,2023-05-22,13,1950.0,2
2,T006,C006,2024-01-22,Completed,2023-06-18,12,12999.0,3
3,T008,C008,2024-01-25,Completed,2023-08-15,17,1299.0,4
4,T009,C009,2024-01-28,Cancelled,2023-09-30,11,4599.0,5
5,T010,C010,2024-02-01,Completed,2023-10-12,10,4495.0,6
6,T011,C011,2024-02-02,Completed,2023-11-05,9,69999.0,7
7,T013,C013,2024-02-08,Completed,2024-01-08,8,1350.0,8
8,T014,C014,2024-10-02,Completed,2024-01-15,7,2998.0,9
9,T015,C015,2024-02-12,Completed,2024-01-20,15,3897.0,10


In [362]:
order_item=sales.merge(
        product,
        how='left',
        on='product_id'
    )



In [365]:
order_items=order_item.merge(
    order,
    how='left',
    on='transaction_id'
)

In [364]:
order

,transaction_id,customer_id,transaction_date,status,registration_date,customer_sk,total_amount,order_sk
0,T002,C002,2024-01-16,Completed,2023-02-20,14,5998.0,1
1,T005,C005,2024-01-20,Completed,2023-05-22,13,1950.0,2
2,T006,C006,2024-01-22,Completed,2023-06-18,12,12999.0,3
3,T008,C008,2024-01-25,Completed,2023-08-15,17,1299.0,4
4,T009,C009,2024-01-28,Cancelled,2023-09-30,11,4599.0,5
5,T010,C010,2024-02-01,Completed,2023-10-12,10,4495.0,6
6,T011,C011,2024-02-02,Completed,2023-11-05,9,69999.0,7
7,T013,C013,2024-02-08,Completed,2024-01-08,8,1350.0,8
8,T014,C014,2024-10-02,Completed,2024-01-15,7,2998.0,9
9,T015,C015,2024-02-12,Completed,2024-01-20,15,3897.0,10


In [372]:
order_items

,transaction_id,customer_id_x,quantity,unit_price_x,transaction_date_x,status_x,product_id,sales_sk,product_name,category,unit_price_y,stock_quantity,product_sk,customer_id_y,transaction_date_y,status_y,registration_date,customer_sk,total_amount,order_sk
0,T002,C002,2,2999.0,2024-01-16,Completed,P004,1,Levi's Jeans,Fashion,2999.0,120.0,3.0,C002,2024-01-16,Completed,2023-02-20,14.0,5998.0,1.0
2,T005,C005,3,650.0,2024-01-20,Completed,P009,3,Basmati Rice 5kg,Groceries,650.0,300.0,7.0,C005,2024-01-20,Completed,2023-05-22,13.0,1950.0,2.0
3,T006,C006,1,12999.0,2024-01-22,Completed,P012,4,Dell Monitor 24inch,Electronics,12999.0,40.0,9.0,C006,2024-01-22,Completed,2023-06-18,12.0,12999.0,3.0
5,T008,C008,1,1299.0,2024-01-25,Completed,P008,6,Adidas T-Shirt,Fashion,1299.0,150.0,6.0,C008,2024-01-25,Completed,2023-08-15,17.0,1299.0,4.0
6,T009,C009,1,4599.0,2024-01-28,Cancelled,P011,7,Puma Sneakers,Fashion,4599.0,70.0,8.0,C009,2024-01-28,Cancelled,2023-09-30,11.0,4599.0,5.0
9,T011,C011,1,69999.0,2024-02-02,Completed,P014,10,iPhone 13,Electronics,69999.0,80.0,11.0,C011,2024-02-02,Completed,2023-11-05,9.0,69999.0,7.0
11,T013,C013,3,450.0,2024-02-08,Completed,P015,12,Organic Honey 500g,Groceries,450.0,200.0,12.0,C013,2024-02-08,Completed,2024-01-08,8.0,1350.0,8.0
12,T014,C014,2,1499.0,2024-10-02,Completed,P019,13,Boat Earbuds,Electronics,1499.0,250.0,15.0,C014,2024-10-02,Completed,2024-01-15,7.0,2998.0,9.0
13,T015,C015,3,1299.0,2024-02-12,Completed,P008,14,Adidas T-Shirt,Fashion,1299.0,150.0,6.0,C015,2024-02-12,Completed,2024-01-20,15.0,3897.0,10.0
14,T017,C017,1,32999.0,2024-02-18,Completed,P016,15,Samsung TV 43inch,Electronics,32999.0,35.0,13.0,C017,2024-02-18,Completed,2024-02-12,5.0,32999.0,11.0


In [371]:
order_items.isna().sum()

transaction_id        0
customer_id_x         0
quantity              0
unit_price_x          0
transaction_date_x    0
status_x              0
product_id            0
sales_sk              0
product_name          0
category              0
unit_price_y          0
stock_quantity        0
product_sk            0
customer_id_y         0
transaction_date_y    0
status_y              0
registration_date     0
customer_sk           0
total_amount          0
order_sk              0
dtype: int64

In [370]:
order_items.dropna(subset=['product_sk'],inplace=True)

In [375]:
order_items = order_items.drop(columns=['customer_id_x', 'customer_id_y','transaction_date_x','status_x','product_name','category','unit_price_y']).reset_index(drop=True)


In [388]:
order_items

,transaction_id,quantity,unit_price_x,product_sk,order_sk,sub_total
0,T002,2,2999.0,3,1,5998.0
1,T005,3,650.0,7,2,1950.0
2,T006,1,12999.0,9,3,12999.0
3,T008,1,1299.0,6,4,1299.0
4,T009,1,4599.0,8,5,4599.0
5,T011,1,69999.0,11,7,69999.0
6,T013,3,450.0,12,8,1350.0
7,T014,2,1499.0,15,9,2998.0
8,T015,3,1299.0,6,10,3897.0
9,T017,1,32999.0,13,11,32999.0


In [377]:
order_items = order_items.drop(columns=['customer_sk', 'registration_date','status_y','transaction_date_y','sales_sk']).reset_index(drop=True)


In [379]:
order_items['stock_quantity'] = order_items['stock_quantity'].astype(int)

In [380]:
order_items['product_sk'] = order_items['product_sk'].astype(int)

In [381]:
order_items['order_sk'] = order_items['order_sk'].astype(int)

In [ ]:

order_items = order_items.drop(columns=['product_id','stock_quantity','total_amount']).reset_index(drop=True)


In [387]:
order_items['sub_total'] = order_items['unit_price_x'] * order_items['quantity']

In [393]:
for ord_it in order_items.to_dict(orient='records'):
    sql = """
        INSERT INTO order_items  (order_id, product_id, quantity, unit_price, subtotal)
        VALUES (%s, %s, %s, %s, %s)
    """
    value = (ord_it['order_sk'], ord_it['product_sk'], ord_it['quantity'], ord_it['unit_price_x'], ord_it['sub_total'])
    cursor.execute(sql, value)
    conn.commit()
    print(cursor.rowcount, ord_it['transaction_id'], "inserted.")

1 T002 inserted.
1 T005 inserted.
1 T006 inserted.
1 T008 inserted.
1 T009 inserted.
1 T011 inserted.
1 T013 inserted.
1 T014 inserted.
1 T015 inserted.
1 T017 inserted.
1 T019 inserted.
1 T020 inserted.
1 T021 inserted.
1 T022 inserted.
1 T024 inserted.
1 T025 inserted.
1 T026 inserted.
1 T028 inserted.
1 T029 inserted.
1 T031 inserted.
1 T033 inserted.
1 T034 inserted.
1 T035 inserted.
1 T036 inserted.
1 T039 inserted.
1 T040 inserted.
